# Requirements to Run
- Scikit-learn version 1.4
- Jupyter / IPython
- pandas
- matplotlib
- seaborn

Assumption - Do we all have conda?

Then, can install with:
```
conda install -c conda-forge scikit-learn=1.4 jupyter pandas seaborn matplotlib
```

If there are issues with the environment (Conflicts, etc), you can create a new environment with:
```
conda create -n ssc-granada -c conda-forge scikit-learn=1.4 umap-learn jupyter pandas seaborn matplotlib
```

In [ ]:
import numpy as np
import pandas as pd

from sklearn import tree
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble
from sklearn import neighbors
from sklearn import neural_network

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.cm as colormap
import seaborn as sns

from IPython.display import Image

# Load in the Data

Note that the data should be split into train and test sets using the script in the bin directory before running this notebook.

When training different models it is important to keep a seperate test set that is not used for training. This is to ensure that the model is not overfitting to the training data. The test set should only be used to evaluate the final model. The test set should not be used to tune the model. If the model is tuned using the test set then the model will not generalise well to unseen data.

In order to evaluate the intermediate models, we can use the validation set. The validation set is used to tune the model's hyperparameters. The validation set is used to select the best model. The validation set is not used to train the model. The validation set should not be used to evaluate the final model.

An alternative approach to the validation set is to use cross validation. Cross validation is a technique that allows us to use all of the data for training and validation. This is useful when we have a small dataset. Cross validation should not be used to evaluate the final model.

In [ ]:
# Switch the data type by changing the path
# 3 Options by default: raw, raw-fft-split-1, raw-fft-split-2
# raw: time series data, 6000 samples, 100 Hz, padded
# raw-fft-split-1: Absolute of FFT of unpadded time series data, 512 bins
# raw-fft-split-3: Split FFT into 3 Sections in Time: Absolute of FFT of each time series chunk: 512 bins per chunk

train_data = np.load("../datasets/llaima/raw-fft-split-3/train_data.npy")
train_labels = np.load("../datasets/llaima/train_labels.npy")

val_data = np.load("../datasets/llaima/raw-fft-split-3/val_data.npy")
val_labels = np.load("../datasets/llaima/val_labels.npy")

test_data = np.load("../datasets/llaima/raw-fft-split-3/test_data.npy")
test_labels = np.load("../datasets/llaima/test_labels.npy")

In [ ]:
# Here we ensure that the data is in the correct format for the models
# Note that we reshape the data to be 1D arrays
# This only affects the case where we have split in time and computed the FFT on multiple segments
train_data = train_data.reshape(train_data.shape[0], -1)
val_data = val_data.reshape(val_data.shape[0], -1)
test_data = test_data.reshape(test_data.shape[0], -1)

In [ ]:
label_dict = {
    0: "LP",
    1: "TC",
    2: "TR",
    3: "VT",
}

## What does the data look like

In [ ]:
grid_size = 3 
fig, ax = plt.subplots(grid_size, grid_size, figsize=(7, 7))
for i in range(grid_size):
    for j in range(grid_size):
        ax[i][j].plot(train_data[i * 5 + j])
        ax[i][j].set_title(label_dict[train_labels[i * 5 + j]])
fig.suptitle("Sample Events")
fig.tight_layout()

In [ ]:
# Average the data for each class
# This is used to visualise the "average" event for each class

# First we need to find the indices of each class
lp_indices = np.where(train_labels == 0)[0]
tc_indices = np.where(train_labels == 1)[0]
tr_indices = np.where(train_labels == 2)[0]
vt_indices = np.where(train_labels == 3)[0]

# Now we can average the data for each class
lp_data = np.mean(train_data[lp_indices], axis=0)
tc_data = np.mean(train_data[tc_indices], axis=0)
tr_data = np.mean(train_data[tr_indices], axis=0)
vt_data = np.mean(train_data[vt_indices], axis=0)

# Plot the average events
fig, axs = plt.subplots(2, 2, figsize=(7, 7))
axs[0][0].plot(lp_data)
axs[0][0].set_title("LP")

axs[0][1].plot(tc_data)
axs[0][1].set_title("TC")

axs[1][0].plot(tr_data)
axs[1][0].set_title("TR")

axs[1][1].plot(vt_data)
axs[1][1].set_title("VT")

for ax in axs.flat:
    ax.set(xlabel='Frequency (nbin)', ylabel='Amplitude')

fig.suptitle("Average Events")
fig.tight_layout()

# ML Models

In [ ]:
# Need to store the performance
validation_performance = []

## K Nearest Neighbor Model

The idea of a K Nearest Neighbour model can be associated with a simple idea of similarity. If we have a new data point, we can find the K most similar data points in the training set. We can then use the labels of these data points to predict the label of the new data point. The label will be determined based on a vote of the K nearest neighbors.

The intuition behind the model is that similar data points will have similar labels. This is a simple model that can be used as a baseline for more complex models. The figure below shows how the method works (credit - Datacamp, https://www.datacamp.com/tutorial/k-nearest-neighbors-knn-classification-with-r-tutorial). With 3 neighbors, the new example will be labelled as a triangle. With 7 neighbors, the new point will be labelled as a star. The model is sensitive to the number of neighbors.

You have seen these type of models within the UMAP class that Michael ran.

In [ ]:
Image(filename='images/knn-example.jpeg')

In [ ]:
# Train the model
neighbors_model = neighbors.KNeighborsClassifier(n_neighbors=5)
neighbors_model.fit(train_data, train_labels)

train_pred = neighbors_model.predict(train_data)
train_acc = metrics.accuracy_score(train_labels, train_pred)

# Predict the validation Set
val_pred = neighbors_model.predict(val_data)
val_acc = metrics.accuracy_score(val_labels, val_pred)
val_acc

print(f"Train Accuracy: {train_acc}, Validation Accuracy: {val_acc}")

In [ ]:
validation_performance.append(
    {"name": "KNN", "accuracy": val_acc, "model": neighbors_model}
)

## Decision Trees

Decision Trees are a simple model that can be used for classification and regression. The model is a tree structure where each node represents a feature and splits the data based on the feature. The model is trained by finding the best feature to split the data on at each node. The model is trained recursively until a stopping criteria is met. The stopping criteria could be a maximum depth of the tree or a minimum number of samples at each node.

A simple example can be seen below to see if someone is fit or not (credits: Datacamp - https://www.datacamp.com/tutorial/decision-tree-classification-python)

In [ ]:
Image(filename='images/dt-example.png', width=500)

Now we can look at training a decision tree for our data!

In [ ]:
dt_model = tree.DecisionTreeClassifier()

# Train the model
dt_model.fit(train_data, train_labels)
train_pred = dt_model.predict(train_data)
train_acc = metrics.accuracy_score(train_labels, train_pred)

# Predict the validation Set
val_pred = dt_model.predict(val_data)
val_acc = metrics.accuracy_score(val_labels, val_pred)

In [ ]:
tree.plot_tree(dt_model)
print(
    f"DT Depth: {dt_model.get_depth()}, DT Number Leaves: {dt_model.get_n_leaves()}, Training Accuracy: {train_acc}, Validation Accuracy: {val_acc}"
)

In [ ]:
validation_performance.append({"name": "DT", "accuracy": val_acc, "model": dt_model})

## DT with Restrictions
The performance of the model on unseen data is not very good... Why? The reason is because the model is overfitting to the training data. This means that the model is fitting to the noise in the training set and not learning the "true" pattern of the data. As a result, it is not generalising to unseen data. This is a common problem in machine learning. We can see that the model is overfitting because the training accuracy is much higher than the validation accuracy.

This overfitting can be combatted by adding restrictions to the decision tree. By limiting the depth, or the number of leaf nodes, we can restrict the capability of the model and prevent it from overfitting. With these restrictions, the performance on the training set is likely to decrease but the difference in performance should decrease and the performance on the validation set should increase.

In [ ]:
dt_rest_model = tree.DecisionTreeClassifier(max_leaf_nodes=30, max_depth=7)
dt_rest_model.fit(train_data, train_labels)
train_pred = dt_rest_model.predict(train_data)
train_acc = metrics.accuracy_score(train_labels, train_pred)

# Predict the validation Set
val_pred = dt_rest_model.predict(val_data)
val_acc = metrics.accuracy_score(val_labels, val_pred)

# Plot the tree and the results
tree.plot_tree(dt_rest_model)
print(
    f"DT Depth: {dt_rest_model.get_depth()}, DT Min Samples per Leaf: {dt_rest_model.get_n_leaves()}, Training Accuracy: {train_acc}, Validation Accuracy: {val_acc}"
)

In [ ]:
validation_performance.append(
    {"name": "DT Restricted", "accuracy": val_acc, "model": dt_rest_model}
)

## Grid Search Parameters
Deciding which restrictions to place is not easy, and depends on the structure of the dataset. There is no one-size fits all solution. We can use a grid search to find the best parameters for our model. The grid search will train a model for each combination of parameters and select the best model based on the validation set. The grid search will then return the best model and the best parameters.

Below we look at a grid search for our decision tree model.

In [ ]:
param_grid = {
    "criterion": ["gini", "entropy"],
    "max_leaf_nodes": list(range(30, 70, 20)),
    "max_depth": list(range(4, 12, 4)),
}


tuned_dt_model = GridSearchCV(
    tree.DecisionTreeClassifier(),
    param_grid,
    cv=5,
    verbose=1,
    return_train_score=True,
    n_jobs=-1,
)

tuned_dt_model.fit(train_data, train_labels)
train_pred = tuned_dt_model.predict(train_data)
train_acc = metrics.accuracy_score(train_labels, train_pred)

# Predict the validation Set
val_pred = tuned_dt_model.predict(val_data)
val_acc = metrics.accuracy_score(val_labels, val_pred)

# Plot the tree and the results
tree_model = tuned_dt_model.best_estimator_
tree.plot_tree(tree_model)
print(
    f"Criterion: {tuned_dt_model.best_params_['criterion']}, DT Depth: {tree_model.get_depth()}, DT Min Samples per Leaf: {tree_model.get_n_leaves()}, Training Accuracy: {train_acc}, Validation Accuracy: {val_acc}"
)

Performance increases!

In [ ]:
validation_performance.append(
    {"name": "DT Tuned", "accuracy": val_acc, "model": tuned_dt_model}
)

# Ensemble Methods

Another group of methods are ensemble methods. These methods combine multiple models together to build a better model. The idea is that the models will learn different aspects of the data and combining them will give a better model. The two main ensemble methods are bagging and boosting.

One example of an ensemble method would be to train multiple models on the data independently and combining the results, generally via majority vote (in the case of classification).

A more "human" example of an ensemble method would be to ask multiple people to classify the data and then combine the results. This is the idea behind the "Who wants to be a millionaire" lifeline. The audience is asked to vote on the answer and the contestant can use this information to help them answer the question.

In [ ]:
Image("images/ensemble.png", width=500)

In general we want to ensure that each of our models has a different perspective on the problem. We can achieve this by training each model on a different subset of the data. This is known as bootstrap-aggregation* or bagging. Bagging can also be used to reduce the variance of a model. This is known as bootstrap aggregation. The idea is that we train multiple models on different subsets of the data and then combine the results. The figure below shows how bagging works (credit - Datacamp, https://www.datacamp.com/community/tutorials/random-forests-classifier-python).

The reason that multiple perspectives are desired is because we want to reduce the correlation between the models. If the models are highly correlated then the ensemble will not perform well. This is because the models will be learning the same thing and will not be able to provide different perspectives.

*bootstrapping refers to the process of sampling with replacement.

In [ ]:
Image("images/bootstrapping.png", width=500)

## Bagging & Random Forests

Random Forest models are an ensemble method that use bagging with multiple decision tree models. Below we train a random forest model that uses bagging and we bootstrap or sample on the features. Note that there are different ways to ensure variance in the decision trees. We could boostrap on the samples. We can restrict the depth of the decision trees, ensuring they do not have the capability to overfit or to learn the relationship perfectly. This should lead to variance within the ensemble

An example of random forest trees is shown below (credit - Datacamp, https://www.datacamp.com/community/tutorials/random-forests-classifier-python). 

In [ ]:
Image("images/random-forest.png", width=500)

In [ ]:
rf_model = ensemble.RandomForestClassifier(
    n_estimators=100, max_features=0.5, max_depth=12, min_samples_leaf=5, n_jobs=-1
)

rf_model.fit(train_data, train_labels)
train_pred = rf_model.predict(train_data)
train_acc = metrics.accuracy_score(train_labels, train_pred)

# Predict the validation Set
val_pred = rf_model.predict(val_data)
val_acc = metrics.accuracy_score(val_labels, val_pred)

# Print the results
print(f"Training Accuracy: {train_acc}, Validation Accuracy: {val_acc}")

In [ ]:
validation_performance.append(
    {"name": "Random Forest", "accuracy": val_acc, "model": rf_model}
)

## Boosting & Gradient Boosted Trees

Boosting is a different class of methods that train models sequentially. The idea is that each model can use the predictions of the previous model. The figure below illustrates the difference between bagging and boosting (credit - Datacamp, https://www.datacamp.com/tutorial/what-bagging-in-machine-learning-a-guide-with-examples).

In [ ]:
Image("images/bagging-boosting.png", width=500)

To make the boosting process clearer, please see the figure below (credit - Datacamp, https://www.datacamp.com/tutorial/ensemble-learning-python). The process used for a common boosting method (AdaBoost) is as follows:
1. Form a large set of simple features
2. Initialize weights for training images
3. For T rounds
    1. Normalize the weights
    2. For available features from the set, train a classifier using a single feature and evaluate the training error
    3. Choose the classifier with the lowest error
    4. Update the weights of the training images: increase if classified wrongly by this classifier, decrease if correctly
4. Form the final strong classifier as the linear combination of the T classifiers (coefficient larger if training error is small)


In [ ]:
Image("images/boosting.png", width=500)

In [ ]:
# Adaboost - NB uses a Decision Tree by default
ada_model = ensemble.AdaBoostClassifier(
    n_estimators=100, learning_rate=0.1
)

ada_model.fit(train_data, train_labels)
train_pred = ada_model.predict(train_data)
train_acc = metrics.accuracy_score(train_labels, train_pred)

# Predict the validation Set
val_pred = ada_model.predict(val_data)
val_acc = metrics.accuracy_score(val_labels, val_pred)

# Prin the results
print(f"Training Accuracy: {train_acc}, Validation Accuracy: {val_acc}")

In [ ]:
validation_performance.append(
    {"name": "AdaBoost", "accuracy": val_acc, "model": ada_model}
)

Another common tree-based ensemble family are gradient-boosted trees. These models are similar to random forests but instead of training multiple decision trees independently, the trees are trained sequentially. The first tree makes a prediction about the data. The second tries to correct the errors of the previous tree. This process continues, up to N trees.

 The idea is that each tree can learn from the previous tree. The figure below offers insights into how the algorithm works (credit - Bobby Tan (Liang Wei), https://towardsdatascience.com/the-intuition-behind-gradient-boosting-xgboost-6d5eac844920).

 For more information on gradient boosted trees, I recommend both the attached article, which goes over the algorithm at a high level and the paper about XGBoost, one of the early gradient-boosted tree methods - https://arxiv.org/pdf/1603.02754.pdf.

In [ ]:
Image("images/gradient-boosting.png", width=500)

In [ ]:
# Here we use the scikit learn implementation of the Gradient Boosting Classifier
gb_model = ensemble.HistGradientBoostingClassifier(
    max_iter=100, max_features=0.5, max_depth=12, min_samples_leaf=5
)

gb_model.fit(train_data, train_labels)
train_pred = gb_model.predict(train_data)
train_acc = metrics.accuracy_score(train_labels, train_pred)

# Predict the validation Set
val_pred = gb_model.predict(val_data)
val_acc = metrics.accuracy_score(val_labels, val_pred)

# Prin the results
print(f"Training Accuracy: {train_acc}, Validation Accuracy: {val_acc}")

In [ ]:
validation_performance.append(
    {"name": "Boosted Trees", "accuracy": val_acc, "model": gb_model}
)

# Multi-Layer Perceptron Neural Networks

The final class of models we will look at are neural networks. Neural networks are a class of models that are inspired by the brain. The idea is that the model is made up of multiple layers of neurons. Each neuron is a simple function that takes in the output of the previous layer and outputs a value. By building a "network" of these neurons, we can build a model that can learn complex relationships between the input and output.

These networks must be trained using a method called backpropagation. This method is a way of calculating the gradient of the loss function with respect to the weights and biases of the network. The gradient is then used to update the neurons of the network. This process is repeated until the loss function converges.

There are many different types of neural networks. The simplest is a multi-layer perceptron (MLP). This is a network that has an input layer, an output layer and one or more hidden layers. The figure below shows a simple MLP (credit - Avinash Navlani, https://machinelearninggeek.com/multi-layer-perceptron-neural-network-using-python/).

In [ ]:
Image("images/mlp.png", width=500)

In [ ]:
mlp_model = neural_network.MLPClassifier(
    hidden_layer_sizes=(400, 100), learning_rate="adaptive"
)

mlp_model.fit(train_data, train_labels)
train_pred = mlp_model.predict(train_data)
train_acc = metrics.accuracy_score(train_labels, train_pred)

# Predict the validation Set
val_pred = mlp_model.predict(val_data)
val_acc = metrics.accuracy_score(val_labels, val_pred)

print(f"Training Accuracy: {train_acc}, Validation Accuracy: {val_acc}")

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3))
ax.plot(mlp_model.loss_curve_)
ax.set_title("Training Loss Curve")
ax.set_ylabel("Training Loss")
ax.set_xlabel("Iterations")

In [ ]:
validation_performance.append({"name": "MLP", "accuracy": val_acc, "model": mlp_model})

# Comparing Models

The final point to discuss here is comparing different models performance. As we have been using accuracy up to this point as evaluation metric, we can continue to do so here *however* please see the note below about metrics!

So first, let's look at the accuracy of the models and select our best model based on this metric.

In [ ]:
df = pd.DataFrame(validation_performance)
df.drop_duplicates(subset="name", inplace=True)
best_model_name = df.loc[df["accuracy"].idxmax()]["name"]
palette = colormap.rainbow(np.linspace(0, 1, len(df)))
colors = ["tab:red" if v == best_model_name else palette[i] for i, v in enumerate(df.name)]
colors = ["tab:red" if v == best_model_name else "tab:blue" for i, v in enumerate(df.name)]
fig, ax = plt.subplots(figsize=(12, 5))
sns.barplot(x="name", y="accuracy", data=df, ax=ax, hue="name", palette=colors)
ax.hlines(
    df["accuracy"].max(),
    -0.5,
    len(df) - 0.5,
    color="tab:red",
    label="Best Model",
    linestyles="dashed",
)

ax.set_title("Validation Accuracy")
ax.set_ylabel("Accuracy")
ax.set_xlabel("Model")

*NB NB NB*

There are other types of metrics that should be considered (Precision, Recall & F1 score). These other metrics are useful for understanding the performance across all classes even in the case of imbalanced datasets. For example, if we have a dataset with 90% of the data in class A and 10% of the data in class B, then a model that always predicts class A will have an accuracy of 90%. This is not a good model. However, if we look at the precision, recall and F1 score, we can see that the model is not performing well. For more information on these useful metrics, please reference the following article - https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9.

Additionally, it is useful to view the confusion matrix and it can be useful to visualise the performance with a confusion matrix plot. This plot illustrates how the model is classifying the samples from different classes and visualises the misclassifications.

We can calulate these metrics for our best model, and also look at the confusion matrix for this model.

In [ ]:
best_model = df.loc[df["accuracy"].idxmax()]["model"]

best_pred = best_model.predict(val_data)

# Calculate metrics
accuracy = metrics.accuracy_score(val_labels, best_pred)
precision = metrics.precision_score(val_labels, best_pred, average="macro")
recall = metrics.recall_score(val_labels, best_pred, average="macro")
f1 = metrics.f1_score(val_labels, best_pred, average="macro")

# Plot the confusion matrix
cm = metrics.confusion_matrix(val_labels, best_pred)
fig, ax = plt.subplots(figsize=(5, 5))
# sns.heatmap(cm, annot=True, fmt="d", ax=ax, cmap="Blues")

metrics.ConfusionMatrixDisplay(cm, display_labels=[label_dict[i] for i in range(4)]).plot(ax=ax, cmap="Blues")
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

# Evaluating Performance on Unseen Data

Now we can take our "best model" and see how it performs on the unseen data. As we have decided on the model, we can now train it on the train and validation set combined. This will result in a better model than if we had trained it on the train set alone. We can then evaluate the new model on the test set.

In [ ]:
# Combine train and validation data
train_val_data = np.concatenate((train_data, val_data))
train_val_labels = np.concatenate((train_labels, val_labels))

# Train the model on the combined data
best_model.fit(train_val_data, train_val_labels)
train_val_pred = best_model.predict(train_val_data)
train_val_acc = metrics.accuracy_score(train_val_labels, train_val_pred)

# Predict the test set
test_pred = best_model.predict(test_data)

# Calculate metrics
accuracy = metrics.accuracy_score(test_labels, test_pred)
precision = metrics.precision_score(test_labels, test_pred, average="macro")
recall = metrics.recall_score(test_labels, test_pred, average="macro")
f1 = metrics.f1_score(test_labels, test_pred, average="macro")

# Plot the confusion matrix
cm = metrics.confusion_matrix(test_labels, test_pred)
fig, ax = plt.subplots(figsize=(5, 5))

metrics.ConfusionMatrixDisplay(cm, display_labels=[label_dict[i] for i in range(4)]).plot(ax=ax, cmap="Blues")
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

# Now What?
Try to beat the performance! Can you improve the model? Can you improve the performance on the test set? Can you improve the performance on the validation set? Can you improve the performance on the training set? Can you improve the performance on the unseen data?

If so, you can win a prize!

_Some tips_
- We only did a grid search on the decision tree model. Can you do a grid search on the other models?
- Can you try different models in the ensembles?
- Could you do a feature transformation and try different models on the transformed data?

